# advent of code 2024 - [day 24](https://adventofcode.com/2024/day/24)


In [29]:
filename = 'input.txt'
#filename = 'test.txt'

In [30]:
def gen_init(file='input.txt'):
    file = open(file, 'r')
    for ix, line in enumerate(file):
        if line.strip() == '':
            break
        wire, input = line.strip().split(': ')
        yield {'wire': wire, 'input': int(input)}
    
def gen_connections(file='input.txt'):
    file = open(file, 'r')
    for ix, line in enumerate(file):
        if ':' in line or line.strip() == '':
            continue
        connection, output_wire = line.strip().split(' -> ')
        wire1, gate, wire2 = connection.strip().split(' ')
        yield {'wire1': wire1, 'wire2': wire2, 'gate': gate, 'output_wire': output_wire}

In [32]:
import os

NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']

from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

from neo4j import GraphDatabase
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

2.13.0


In [33]:
# cleaning
def clean():
    queries = [
    "CALL apoc.schema.assert({},{});",
    """MATCH (n)
    CALL {WITH n DETACH DELETE n}
    IN TRANSACTIONS OF 1000 ROWS;""",
    """CALL gds.graph.list()
    YIELD graphName
    WITH graphName AS g
    CALL (g) {CALL gds.graph.drop(g) YIELD graphName RETURN graphName}
    WITH graphName RETURN graphName;"""]

    for q in queries:
        gds.run_cypher(q, {})

In [34]:
def ingest(filename):

    initial_values = list(gen_init(filename))
    connections = list(gen_connections(filename))

    
    clean()

    gds.run_cypher('CREATE INDEX wire_id IF NOT EXISTS FOR (w:Wire) ON (w.id)')
    
    ingest_qs = ["""
    UNWIND $connections AS c
    MERGE (w1:Wire {id:c.wire1})
    MERGE (w2:Wire {id:c.wire2})
    MERGE (out_w:Wire {id:c.output_wire})
    CREATE (g:Gate)
    SET g:$(c.gate)
    CREATE (w1)-[:IN]->(g), (w2)-[:IN]->(g), (g)-[:OUT]->(out_w)
    """,
    """
    UNWIND $initial_values AS val
    MERGE (w:Wire {id:val.wire})
    SET w:Processed, w.bit = val.input
    """]

    for q in ingest_qs:
        gds.run_cypher(q, {"connections":connections, "initial_values": initial_values})

In [35]:
ingest(filename)

In [36]:
def infer (rules, params={}):
    """
    This is a function you can use if you want to run a set of inference rules
    until a convergence is reached. why not use it in a RDF-like reasoning context?
    """
    counter = 0
    while True:
        counter += 1
        any_update = False
        for rule in rules:
            with driver.session(database="neo4j") as session:
                result = session.run(rule, params)
            any_update = any_update or result.consume().counters._contains_updates
        if not any_update:
            break

In [37]:
rules = ["""MATCH (in1:Wire&Processed {bit:1})-[:IN]->(g:Gate&OR)-[:OUT]->(out:Wire&!Processed)
SET out:Processed, out.bit = 1""",
"""MATCH (in1:Wire&Processed {bit:0})-[:IN]->(g:Gate&OR)-[:OUT]->(out:Wire&!Processed), (in2:Wire&Processed {bit:0})-[:IN]->(g)
SET out:Processed, out.bit = 0""",
"""MATCH (in1:Wire&Processed {bit:0})-[:IN]->(g:Gate&AND)-[:OUT]->(out:Wire&!Processed)
SET out:Processed, out.bit = 0""",
"""MATCH (in1:Wire&Processed {bit:1})-[:IN]->(g:Gate&AND)-[:OUT]->(out:Wire&!Processed), (in2:Wire&Processed {bit:1})-[:IN]->(g)
SET out:Processed, out.bit = 1""",
"""MATCH (in1:Wire&Processed)-[:IN]->(g:Gate&XOR)-[:OUT]->(out:Wire&!Processed), (in2:Wire&Processed)-[:IN]->(g)
SET out:Processed, out.bit = (CASE in1.bit <> in2.bit WHEN TRUE THEN 1 ELSE 0 END)"""]

infer(rules)

In [38]:
gds.run_cypher("""MATCH (w:Wire) WHERE w.id STARTS WITH 'z'
WITH w ORDER BY w.id DESC
WITH collect(w.bit) AS bits
RETURN reduce(val = 0, b IN bits | val * 2 + b) AS part1""")

,part1
0,51745744348272


### Part 2
by hand